In [5]:
import pandas as pd
import tushare as ts 
import numpy as np   
import more_bisect as mb  
import datetime

from pandas.core.common import flatten 

token = 'c07b6020d5625582f589051513fcf8fc9809e2a0bdc411296b074f4c'
ts.set_token(token) #设置token，只需设置一次
pro = ts.pro_api(timeout=5) #初始化接口  

# start_date = '20150101'
# end_date = '20221227' 
# freq = 'D'
# fq_type = 'qfq' 
# code = '300002.SZ'

# hfq_df = ts.pro_bar(ts_code=code, adj=fq_type, start_date=start_date, end_date=end_date)  
 
def get_data(code, freq, fq_type, start_date, end_date, indexs): 
    df = ts.pro_bar(ts_code=code, freq=freq, adj=fq_type, start_date=start_date, end_date=end_date) 
    df.index = pd.to_datetime(df['trade_date'])   
    df.index.name = None
    df.sort_index(inplace=True) 
    
    return df[[indexs]]  

def show_pdf(df):
    import seaborn as sns 
    sns.set() 
    sns.distplot(df) 

def show_percentile(df, percent):
    print(df.describe(percentiles=[percent]))

if __name__=="__main__": 
    
    token = 'c07b6020d5625582f589051513fcf8fc9809e2a0bdc411296b074f4c'
    ts.set_token(token) #设置token，只需设置一次
    pro = ts.pro_api(timeout=5) #初始化接口  

    start_dt = '20220101'  
    end_dt = datetime.datetime.now().strftime(r"%Y%m%d") 

    # today.strftime(r"%Y%m%d") 
    # end_dt = '20230109'   

    freq = 'D'
    fq_type = 'qfq' 
    code = '688031.SH'
    indexs = 'close' 
    percent = 0.8

    df = get_data(code, freq, fq_type, start_dt, end_dt, indexs)  

    # 统计收盘价的分布，自定义百分位数
    stat = df.describe(percentiles=[percent])    

    # 统计滑动窗的上下限
    pri_low, pri_high = stat.loc['min'].values, stat.loc[format(percent, ".0%")].values  

    # 设定档位数，同时求出对应的步长  
    stage = 9
    step = (pri_high - pri_low) / stage 

    # winDown 是下行操作字典， winUp是上行操作字典
    winDown = np.around(np.linspace(pri_high, pri_low, stage), decimals=1).flatten() 
    winDown = np.flip(winDown)  # 下行操作字典也是升序，保证可以使用二分搜索找到操作值 

    winDown_ext = np.append(winDown, 0)   # 价格区间切分点 

    winUp = np.around(np.linspace(pri_low+step, pri_high+step, stage), decimals=1).flatten()   

    # Show pdf and histgram of close price of every day 
    # show_pdf(df)  

    # Show percentile of dataset 
    # show_percentile(df, percent)

    # 测试df数据格式 
    # print(df.tail())   

    price = df.close  # 从 DF 中提取价格数据 

    valid_price = price[price<winDown.max()]   # 筛选出小于winDown最大值的价格，作为有效价格
    tmp = valid_price[::] 
    price_pair = zip(tmp, tmp[1:])  # 价格对（滑动窗，步进）

    tmp_op = []  
    In_op = [] 
    Out_op = [] 
    profit = 0

    # 组合成 [later, front] price pair 
    for front, later in price_pair:
        
        # print(front, later) 
        # print(front)  
        # print(later)
        # print("-" * 20) 
        
        if(later < front):  # later小于front, 下跌 
            inOp = winDown_ext[mb.first_pos_ge(later, winDown)]  # (下跌时)买入操作
            if(len(In_op) == 0):
                # tmp_op.append(inOp)
                In_op.append(inOp)   
            elif(len(In_op) != 0 and inOp >= In_op[-1]): 
                # 若In_op非空（非首次买入），且 本次买入档位 大于等于 上次档位，则不执行买入
                continue
            elif(len(In_op) != 0 and inOp < In_op[-1]): 
                # 若In_op非空（非首次买入），且 本次买入档位 小于 上次档位，则执行买入（只买跌）
                In_op.append(inOp) 
                
        elif(later > front):  # later 大于 front, 上涨
            # outOp = winUp[mb.last_pos_le(later, winUp)]    # (上涨时)卖出操作  
            outOp = winUp[mb.last_pos_lt(later, winUp)] if mb.last_pos_lt(later, winUp) is not None else 0 
            if(len(In_op)==0):
                continue
            elif(len(In_op)!=0 and outOp > In_op[-1]):
                Out_op.append(outOp) 
                print(f"In stack before this sell is : {In_op}")
                print(f"Out stack is : {Out_op}")
                print(f"Profit before this sell is {profit}") 
                
                inOp_top = In_op.pop() 
                profit += outOp - inOp_top   # 卖出 - 买入 = 实盈 
                print(f"In stack after this sell is : {In_op}")  
                print(f"Profit after this sell is {profit}") 
                print("--" * 30)
            elif(len(In_op)!=0 and outOp <= In_op[-1]):
                continue  
        
        elif(later == front):
            continue


In stack before this sell is : [72.9, 69.0, 65.0]
Out stack is : [68.5]
Profit before this sell is 0
In stack after this sell is : [72.9, 69.0]
Profit after this sell is 3.5
------------------------------------------------------------
In stack before this sell is : [72.9, 69.0]
Out stack is : [68.5, 80.4]
Profit before this sell is 3.5
In stack after this sell is : [72.9]
Profit after this sell is 14.900000000000006
------------------------------------------------------------
In stack before this sell is : [72.9]
Out stack is : [68.5, 80.4, 80.4]
Profit before this sell is 14.900000000000006
In stack after this sell is : []
Profit after this sell is 22.400000000000006
------------------------------------------------------------
In stack before this sell is : [80.8, 76.9]
Out stack is : [68.5, 80.4, 80.4, 80.4]
Profit before this sell is 22.400000000000006
In stack after this sell is : [80.8]
Profit after this sell is 25.900000000000006
--------------------------------------------------

In [48]:
wd = np.around(np.linspace(pri_high, pri_low, stage), decimals=1).flatten()  
print(wd)

amt = 0.1 
new_wd = [0] * len(wd)
new_wd[0] = wd[0] 

for idx, i in enumerate(wd[1:]):
    new_wd[idx+1] = np.around(wd[idx] * (1-amt), decimals=3) 
     
    # print(idx) 
    # print(i) 
    # print("--" * 20)

print(new_wd) 
# wd_ = [  for i in wd[1:]]  
# b = np.array(a)   



[25.8 25.2 24.5 23.8 23.1 22.4 21.7 21.1 20.4]
[25.8, 23.22, 22.68, 22.05, 21.42, 20.79, 20.16, 19.53, 18.99]


### 0109 Construct price-fractions pair using dictionary

In [ ]:
import more_bisect as mb  
from pandas.core.common import flatten  

price = df.close 

valid_price = price[price<winDown.max()]  # 筛选出小于价格最大值的价格，作为有效价格
tmp = valid_price[::] 
price_pair = zip(tmp, tmp[1:])

tmp_op = []  
In_op = [] 
Out_op = [] 
profit = 0

# 组合成 [later, front] price pair 
for front, later in price_pair:
       
    # print(front, later) 
    # print(front)  
    # print(later)
    # print("-" * 20) 
    
    if(later < front):  # later小于front, 下跌 
        inOp = winDown_ext[mb.first_pos_ge(later, winDown)]  # (下跌时)买入操作
        if(len(In_op) == 0):
            # tmp_op.append(inOp)
            In_op.append(inOp)   
        elif(len(In_op) != 0 and inOp >= In_op[-1]): 
            # 若In_op非空（非首次买入），且 本次买入档位 大于等于 上次档位，则不执行买入
            continue
        elif(len(In_op) != 0 and inOp < In_op[-1]): 
            # 若In_op非空（非首次买入），且 本次买入档位 小于 上次档位，则执行买入（只买跌）
            In_op.append(inOp) 
            
    elif(later > front):  # later 大于 front, 上涨
        # outOp = winUp[mb.last_pos_le(later, winUp)]    # (上涨时)卖出操作  
        outOp = winUp[mb.last_pos_lt(later, winUp)] if mb.last_pos_lt(later, winUp) is not None else 0 
        if(len(In_op)==0):
            continue
        elif(len(In_op)!=0 and outOp > In_op[-1]):
            Out_op.append(outOp) 
            print(f"In stack before this sell is : {In_op}")
            print(f"Out stack is : {Out_op}")
            print(f"Profit before this sell is {profit}") 
            
            inOp_top = In_op.pop() 
            profit += outOp - inOp_top   # 卖出 - 买入 = 实盈 
            print(f"In stack after this sell is : {In_op}")  
            print(f"Profit after this sell is {profit}") 
            print("--" * 30)
        elif(len(In_op)!=0 and outOp <= In_op[-1]):
            continue  
        
    elif(later == front):
        continue
        

#### InOp list with operation amout 

In [3]:
a = [1,2,3] 
amount = 100 
a.extend([3.2] * amount) 
print(a)  


[1, 2, 3, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2]


In [2]:
print(winDown) 
print(winDown_ext)
# print(stage)

amo = np.arange(100*(stage+2), 100, step=100)
winDown_ext_amo = dict(zip(winDown_ext, amo))  
print(winDown_ext_amo) 


NameError: name 'winDown' is not defined

### 0118 Tushare traversed the stock library 

In [30]:
import tushare as ts 
import pandas as pd
import tushare as ts 
import numpy as np   
import more_bisect as mb  
import datetime

from pandas.core.common import flatten 

def init_api():
    token = 'c07b6020d5625582f589051513fcf8fc9809e2a0bdc411296b074f4c'
    ts.set_token(token) #设置token，只需设置一次
    pro = ts.pro_api(timeout=5) #初始化接口  
    return pro 

pro = init_api()
data = pro.query('stock_basic', exchange='', list_status='L', 
                 fields='ts_code,symbol,name,area,industry,list_date') 
# print(data.head())  

valid_data = data[~data['name'].str.startswith('ST')]   
valid_ts_code = valid_data['ts_code']  # 所有非ST股票代码 list

# print(valid_ts_code.values)   


<class 'str'>
000001.SZ
----------------------------------------
<class 'str'>
000002.SZ
----------------------------------------
<class 'str'>
000006.SZ
----------------------------------------
<class 'str'>
000007.SZ
----------------------------------------
<class 'str'>
000008.SZ
----------------------------------------
<class 'str'>
000009.SZ
----------------------------------------
<class 'str'>
000010.SZ
----------------------------------------
<class 'str'>
000011.SZ
----------------------------------------
<class 'str'>
000012.SZ
----------------------------------------
<class 'str'>
000014.SZ
----------------------------------------
<class 'str'>
000016.SZ
----------------------------------------
<class 'str'>
000017.SZ
----------------------------------------
<class 'str'>
000019.SZ
----------------------------------------
<class 'str'>
000020.SZ
----------------------------------------
<class 'str'>
000021.SZ
----------------------------------------
<class 'str'>
000023.SZ
-

#### Computing cost and plans

In [71]:
import numpy as np  
import pandas as pd   
import tushare as ts 

stage1 = 5
power1 = np.logspace(1, stage1, num=stage1, base=0.94) 
price1 = np.round(25.50 * power1, decimals=2)    
price1 = np.insert(price1, 0, 25.50)

stage2 = 6
power2 = np.logspace(1, stage2, num=stage2, base=0.96) 
price2 = np.round(price1[-1] * power2, decimals=2)  
price = np.append(price1, price2) 

# price2 = np.insert(price2, 0, 25.50)  
# print(price2) 

ratio = 4
num_1 = ratio * np.array([100, 200, 300])  
num_2 = ratio * np.array([100, 200, 300])
number = ratio * np.array([100, 200, 400, 800]) 

# numpy array 自我复制的两种方法 np.repeat() np.tile() 
# num_2 = np.tile(number, 2)  
nums = np.concatenate((num_1, num_2, num_2, num_2)) 
print(nums) 
print(price)

cost = np.sum(np.dot(price, nums)) 
print(cost) 

arr_inv = np.concatenate([price, nums.T]) 
print(arr_inv) 

# print(price) 

[ 400  800 1200  400  800 1200  400  800 1200  400  800 1200]
[25.5  23.97 22.53 21.18 19.91 18.71 17.96 17.24 16.55 15.89 15.26 14.65]
180244.0
[  25.5    23.97   22.53   21.18   19.91   18.71   17.96   17.24   16.55
   15.89   15.26   14.65  400.    800.   1200.    400.    800.   1200.
  400.    800.   1200.    400.    800.   1200.  ]


In [43]:
price

array([25.5 , 23.72, 22.05, 20.51, 19.08, 17.74, 17.03, 16.35, 15.7 ,
       15.07])